In [1]:
import numpy as np
import math
import random

In [6]:
class Network:
    #initiating our network
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [ np.random.randn(y,1) for y in sizes[1:] ]
        self.weights = [ np.random.randn(y,x) for y,x in zip(sizes[1:],sizes[:-1])]
        
    def forward(self,a):
        #return w.x + b that is apply the weights and biases
        for w,b in zip(self.weights, self.biases):
            a = sigmoid(np.dot(w,a) + b)
    
    '''
        the training data is a list of tuples (x,y) with inputs and outputs
    '''
    
    def StochasticGradientDesecent(self, training_data, epochs, batch_size, learning_rate, test_data=None ):
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        #here we have used xrange to because it generates number one by one instead of
        #range method which stores a whole list of numbers then iterate
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[ind:ind+batch_size] for ind in xrange(0,n,batch_size)
            ]
            for mini_batch in mini_batches:
                #the function below will update the wieghts and biases based on mini batch and the learning rate
                self.updateParameters(mini_batch, learning_rate)
            if test_data:
                #it will evaluate the function for each epoch - slowing down the training
                pass
                #currently not implmenting
            else:
                print(f"Epoch {j} completed")
    
    def updateParameters(self, batch, eta):
        #nabla represents the gradient operator
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeroes(w.shape) for w in self.weights]
        for x,y in batch:
            #finding derivative for each training example in the batch
            delta_b = self.backprop(x,y)
            delta_w = self.backprop(x,y)
            #adding all the derivative first to take the average derivative later
            nalba_b = [nb + b for nb,b in zip(nabla_b, delta_b)]
            nalba_w = [nw + w for nw,w in zip(nabla_w, delta_w)]
        #updating all the weights
        self.weights = [w-(nabla_w/len(batch))*eta for w in self.weights]
        #updating all the biases
        self.biases = [b-(nabla_b/len(batch))*eta for b in self.biases]
    
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))